# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [8]:
!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt -d data/

mkdir: data: File exists
--2023-10-07 22:13:17--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip [переход]
--2023-10-07 22:13:17--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 4723095 (4.5M) [application/zip]
Сохранение в: «data/corpus_wsd_50k.txt.zip»

corpus_wsd_50k.txt. 100%[===================>]   4.5

In [139]:
corpus_wsd = []
corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [140]:
corpus_wsd

[[['', 'how', 'How'],
  ['long%3:00:02::', 'long', 'long'],
  ['', 'have', 'has'],
  ['', 'it', 'it'],
  ['be%2:42:03::', 'be', 'been'],
  ['', 'since', 'since'],
  ['', 'you', 'you'],
  ['review%2:31:00::', 'review', 'reviewed'],
  ['', 'the', 'the'],
  ['objective%1:09:00::', 'objective', 'objectives'],
  ['', 'of', 'of'],
  ['', 'you', 'your'],
  ['benefit%1:21:00::', 'benefit', 'benefit'],
  ['', 'and', 'and'],
  ['service%1:04:07::', 'service', 'service'],
  ['program%1:09:01::', 'program', 'program'],
  ['', '?', '?']],
 [['', 'have', 'Have'],
  ['', 'you', 'you'],
  ['permit%2:41:00::', 'permit', 'permitted'],
  ['', 'it', 'it'],
  ['', 'to', 'to'],
  ['become%2:42:01::', 'become', 'become'],
  ['', 'a', 'a'],
  ['giveaway%1:21:00::', 'giveaway', 'giveaway'],
  ['program%1:09:01::', 'program', 'program'],
  ['rather%4:02:02::', 'rather', 'rather'],
  ['', 'than', 'than'],
  ['', 'one', 'one'],
  ['', 'that', 'that'],
  ['have%2:42:00::', 'have', 'has'],
  ['', 'the', 'the'],
  [

In [21]:
import nltk 
from nltk.wsd import lesk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import wordnet as wn

In [188]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('English'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ignatenkodarja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [216]:
#_попытка_ имплементировать алгоритм Леска + jaccard score в качестве метрики
def get_semantic(context, key_word): 
    jaccard = -1
    for syn in wn.synsets(key_word):
        definition = syn.definition().split()
        definition = [wn.morphy(token) for token in definition if token not in stopwords]
        intersection = (set(definition) & set(context))
        union = (set(definition) | set(context))
        jaccard_temp =  len(intersection) / len(union)
        if jaccard_temp > jaccard:
            jaccard = jaccard_temp
            res_synset = syn 
    return res_synset

In [257]:
#_попытка_ имплементировать алгоритм Леска + косинусная близость в качестве метрики
def get_semantic2(context, key_word): 
    cosine = -99999
    for syn in wn.synsets(key_word):
        definition = syn.definition().split()
        definition = [wn.morphy(token) for token in definition if token not in stopwords]
        c_vect =[]
        d_vect =[]
        # form a set containing keywords of both strings  
        union = (set(definition) | set(context))
        for w in union: 
            if w in context: 
                c_vect.append(1) # create a vector 
            else: 
                c_vect.append(0) 
            if w in definition: 
                d_vect.append(1) 
            else: 
                d_vect.append(0) 
        c = 0
        # cosine formula  
        for i in range(len(union)): 
            c+= c_vect[i]*d_vect[i]
        try:
            cosine_temp = c / float((sum(c_vect)*sum(d_vect))**0.5)
        except ZeroDivisionError:
            continue
        if cosine_temp > cosine:
            res_synset = syn
            cosine = cosine_temp
    return res_synset

In [265]:
corpus_cosine = []
for s in corpus_wsd:
    corpus_cosine.append(analyze_sentence(s))

In [268]:
#проходится по предложению и для каждого многозначного слова запускает get_semantic(context, key_word) /get_semantic2(context, key_word)
def analyze_sentence(sent):
    #sentence = [token[2] for token in sent if token[2] not in stopwords]
    sent_res = []
    for w in sent:
        if len(w)< 3:
            continue
        w_res = [w[0], w[1], w[2],'']
        if w_res[0] != '':
            w_res[0] = wn.synset_from_sense_key(w_res[0]).name()
        #w_res = [wn.synset_from_sense_key(w[0]).name(), w[1], w[2],'']
        context = []
        lemma = w_res[1]
        #lemma = wn.morphy(w[2].lower()) # или можно просто взять w[1], а не доставать лемму самостоятельно?
        if len(wn.synsets(lemma)) == 1:
            continue
        elif len(wn.synsets(lemma)) > 1: 
            context = [token[2] for token in sent if token[2] not in stopwords and token[2] != w[2]]
            #print(context, '....', lemma)
            #print(get_semantic(context, lemma))
            temp = get_semantic(context, lemma)
            #temp = get_semantic2(context, lemma)
            w_res[3] = temp.name()
        sent_res.append(w_res) 
    return sent_res
            

In [247]:
sent = corpus_wsd[3]

In [248]:
print(analyze_sentence(sent))

[['', 'do', 'Do', 'suffice.v.01'], ['', 'you', 'you', ''], ['measure.v.04', 'measure', 'measure', 'measure.n.01'], ['', 'relation', 'relation', 'relation.n.01'], ['', 'to', 'to', ''], ['decreased.a.01', 'reduced', 'reduced', 'reduce.v.01'], ['', ',', ',', ''], ['employee_turnover.n.01', 'turnover', 'turnover', 'dollar_volume.n.01'], ['', ',', ',', ''], ['accident.n.01', 'accident', 'accidents', 'accident.n.01'], ['', ',', ',', ''], ['', 'and', 'and', ''], ['grievance.n.02', 'grievance', 'grievances', 'grudge.n.01'], ['', ',', ',', ''], ['', 'and', 'and', ''], ['', 'to', 'to', ''], ['improved.s.02', 'improved', 'improved', 'improved.s.02'], ['quality.n.01', 'quality', 'quality', 'quality.n.01'], ['', 'and', 'and', ''], ['output.n.02', 'output', 'output', 'end_product.n.01'], ['', '?', '?', '']]


In [269]:
corpus_jaccard = []
for s in corpus_wsd:
    corpus_jaccard.append(analyze_sentence(s))

In [270]:
corpus_jaccard

[[['', 'how', 'How', ''],
  ['long.a.01', 'long', 'long', 'hanker.v.01'],
  ['', 'have', 'has', 'rich_person.n.01'],
  ['be.v.01', 'be', 'been', 'beryllium.n.01'],
  ['', 'since', 'since', ''],
  ['', 'you', 'you', ''],
  ['review.v.01', 'review', 'reviewed', 'review.n.04'],
  ['', 'the', 'the', ''],
  ['aim.n.02', 'objective', 'objectives', 'aim.n.02'],
  ['', 'of', 'of', ''],
  ['', 'you', 'your', ''],
  ['benefit.n.01', 'benefit', 'benefit', 'benefit.n.01'],
  ['', 'and', 'and', ''],
  ['service.n.05', 'service', 'service', 'service.n.01'],
  ['program.n.02', 'program', 'program', 'plan.n.01'],
  ['', '?', '?', '']],
 [['', 'have', 'Have', 'have.v.17'],
  ['', 'you', 'you', ''],
  ['let.v.01', 'permit', 'permitted', 'license.n.01'],
  ['', 'to', 'to', ''],
  ['become.v.02', 'become', 'become', 'become.v.01'],
  ['', 'a', 'a', 'deoxyadenosine_monophosphate.n.01'],
  ['giveaway.n.01', 'giveaway', 'giveaway', 'game_show.n.01'],
  ['program.n.02', 'program', 'program', 'plan.n.01'],
  [

In [263]:
corpus_cosine

[[['', 'how', 'How', ''],
  ['long.a.01', 'long', 'long', 'hanker.v.01'],
  ['', 'have', 'has', 'rich_person.n.01'],
  ['be.v.01', 'be', 'been', 'beryllium.n.01'],
  ['', 'since', 'since', ''],
  ['', 'you', 'you', ''],
  ['review.v.01', 'review', 'reviewed', 'review.n.04'],
  ['', 'the', 'the', ''],
  ['aim.n.02', 'objective', 'objectives', 'aim.n.02'],
  ['', 'of', 'of', ''],
  ['', 'you', 'your', ''],
  ['benefit.n.01', 'benefit', 'benefit', 'benefit.n.01'],
  ['', 'and', 'and', ''],
  ['service.n.05', 'service', 'service', 'service.n.01'],
  ['program.n.02', 'program', 'program', 'plan.n.01'],
  ['', '?', '?', '']],
 [['', 'have', 'Have', 'have.v.17'],
  ['', 'you', 'you', ''],
  ['let.v.01', 'permit', 'permitted', 'license.n.01'],
  ['', 'to', 'to', ''],
  ['become.v.02', 'become', 'become', 'become.v.01'],
  ['', 'a', 'a', 'deoxyadenosine_monophosphate.n.01'],
  ['giveaway.n.01', 'giveaway', 'giveaway', 'game_show.n.01'],
  ['program.n.02', 'program', 'program', 'plan.n.01'],
  [

In [224]:
def accuracy(corpus):
    T = 0
    F = 0
    for sent in corpus:
        for w in sent:
            if w[0] != '':
                continue
            elif w[0] == w[3]:
                T += 1
            else:
                F += 1
    acc = T / (T + F)
    return acc

In [271]:
accuracy(corpus_jaccard)

0.4904342032752367

In [267]:
accuracy(corpus_cosine)

0.4904342032752367

In [ ]:
#это нормально, что они одинаковые? странно + подозрительно

# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [272]:
!mkdir data
!wget https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv -P data

mkdir: data: File exists
--2023-10-09 20:09:33--  https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 375759 (367K) [text/plain]
Сохранение в: «data/train.csv»

train.csv           100%[===================>] 366.95K  12.7KB/s    за 17s     

2023-10-09 20:09:52 (22.0 KB/s) - «data/train.csv» сохранён [375759/375759]



In [1]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
import numpy as np
from sklearn.metrics import adjusted_rand_score

from IPython.display import Image
from IPython.core.display import HTML
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

Defaulting to user installation because normal site-packages is not writeable
     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/135.3 MB 15.2 kB/s eta 2:24:20

In [ ]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

In [4]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [5]:
for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

    word                                            context  gold_sense_id
383  бор  бор ( элемент ) бор — элемент тринадцатой груп...              1
384  бор  бор - углерод - кремний семейство сплавов на о...              1
385  бор  с большим выделением теплоты , образуется окси...              1
386  бор  это объясняется прежде всего тем , что у компл...              1
387  бор  совсем малочисленна . элементарный бор в приро...              1
388  бор  действующем при месторождении горно - химическ...              1
389  бор  b c ) . при нагревании в атмосфере кислорода и...              1
390  бор  собственных минералов бора в чужих минералах о...              1
391  бор  бор - углерод - кремний семейство сплавов на о...              1
392  бор  с другими галогенами с образованием тригалоген...              1
393  бор  и в стабильны и входят в состав природного бор...              1
394  бор  номером . обозначается символом b ( ) . в своб...              1
395  бор  и взаимные пере

In [6]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(3)
#     cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

NameError: name 'embed' is not defined